# 1. Tester et évaluer un modèle entraîné sur Google News
a. Installez gensim, une librairie Python qui fournit des outils pour travailler avec Word2Vec (avec conda ou avec pip). Prenez la version 3.8.3, et non pas la nouvelle version 4.0.X. 

Obtenez depuis gensim le modèle word2vec pré-entraîné sur le corpus Google News en écrivant : w2v_model = gensim.downloader.load("word2vec-google-news-300"), ce qui téléchargera le fichier la première fois. Ne gardez en mémoire que les vecteurs des mots, en écrivant : « w2v_vectors = w2v_model.wv » puis « del w2v_model ».

• Une fois que vous avez téléchargé le modèle, vous pouvez utiliser votre copie locale : w2v_vectors = KeyedVectors.load_word2vec_format(path_to_file, binary=True).

In [1]:
from gensim import models, downloader

In [2]:
# w2v_model = downloader.load("word2vec-google-news-300")

In [3]:
from gensim.models import KeyedVectors

path_to_file = '~/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz'
w2v_vectors = KeyedVectors.load_word2vec_format(path_to_file, binary=True)

b. Quelle place mémoire occupe le processus du notebook une fois les vecteurs de mots chargés ?

Environ 4GB de RAM

c. Quelle est la dimension de l’espace vectoriel dans lequel les mots sont représentés ?

In [4]:
print(w2v_vectors.vector_size)

300


d. Quelle est la taille du vocabulaire du modèle ? Affichez cinq mots (anglais) qui sont dans le vocabulaire et deux qui ne le sont pas.

In [5]:
print(w2v_vectors.vectors.shape)

# La taille du vocabulaire est de 3'000'000 de mots

(3000000, 300)


In [6]:
words = ['cat', 'bob', 'knife', 'sea', 'lol', 'yolo', 'cuck']

for word in words:
    print(f"Is {word} in the vocabulary? {w2v_vectors.has_index_for(word)}")

Is cat in the vocabulary? True
Is bob in the vocabulary? True
Is knife in the vocabulary? True
Is sea in the vocabulary? True
Is lol in the vocabulary? True
Is yolo in the vocabulary? False
Is cuck in the vocabulary? False


e. Quelle est la distance entre les mots rabbit et carrot ? Veuillez aussi expliquer en une phrase comment on mesurer les distances entre deux mots dans cet espace.

In [7]:
w2v_vectors.distance('rabbit', 'carrot')

# La distance entre deux mots est mesurée en utilisant la similarité cosinus. En effet, la distance correspond à 1 - la similarité.

0.6369357705116272

f. Considérez au moins 5 paires de mots, certains proches par leurs sens, d’autres plus éloignés. Pour chaque paire, calculez la distance entre les deux mots. Veuillez indiquer si les distances obtenues correspondent à vos intuitions sur la proximité des sens des mots.

In [8]:
word_pairs = [('festival', 'watch'), ('confident', 'cocky'), ('tree', 'leaf'), ('meticulous', 'nitpicky'), ('dog', 'car')]
for pair in word_pairs:
    print(f'Distance between {pair[0]} and {pair[1]} : {w2v_vectors.distance(pair[0], pair[1])}')

Distance between festival and watch : 0.9337479248642921
Distance between confident and cocky : 0.6400514543056488
Distance between tree and leaf : 0.517714649438858
Distance between meticulous and nitpicky : 0.704185277223587
Distance between dog and car : 0.6899985373020172


Certaines paires obtiennent des distances qui correspondent à nos intuitions, d'autres non. Par exemple, les paires (festival, watch) et (dog, car) ont une distance élevée car les mots n'ont pas de sens proche. 

Par contre, d'autres paires comme (tree, leaf), (confident, cocky) ou (meticulous, nitpicky) ont une distance plus élevée que ce que nous aurions pensé.

g. Pouvez-vous trouver des mots de sens opposés mais qui sont proches dans l’espace vectoriel ? Comment expliquez vous cela ? Est-ce une qualité ou un défaut du modèle word2vec ?

In [9]:
opposite_pairs = [('good', 'bad'), ('up', 'down'), ('black', 'white')]

for pair in opposite_pairs:
    print(f'Distance entre {pair[0]} et {pair[1]} : {w2v_vectors.distance(pair[0], pair[1])}')

Distance entre good et bad : 0.2809947729110718
Distance entre up et down : 0.36030077934265137
Distance entre black et white : 0.1907784342765808


Cela est dû au fait que les mots peuvent être utilisés dans le même contexte et possèdent un lien sémantique. Il s'agit d'une qualité du modèle car il arrive alors à faire des rapprochements plus subtiles que simplement trouver des similarités entre synonymes par exemple.

h. En vous aidant de la documentation de Gensim sur KeyedVectors, calculez le score du modèle word2vec sur les données WordSimilarity-353. (La doc vous permettra aussi de récupérer le fichier.) Expliquez en 1-2 phrases comment ce score est calculé et ce qu’il mesure.

In [ ]:
# TODO

i. En vous aidant de la documentation, calculez le score du modèle word2vec sur les données questions-words.txt. Attention, cette évaluation prend une dizaine de minutes. Expliquez en 1-2 phrases comment ce score est calculé et ce qu'il mesure.